In [21]:
import pandas as pd
import yfinance as yf
import os

Get the Data

In [10]:
base_dir = '/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/'
index_values_file = 'nifty50_index_prices_2021_2025.csv'
index_cons_file = 'nifty50_cons.csv'

In [6]:
index_values = pd.read_csv(base_dir + index_values_file, usecols=['Date', 'Adj Close'], parse_dates=['Date'])
index_values

,Date,Adj Close
0,2021-01-01,14018.500000
1,2021-01-04,14132.900391
2,2021-01-05,14199.500000
3,2021-01-06,14146.250000
4,2021-01-07,14137.349609
...,...,...
1230,2025-12-23,26177.150391
1231,2025-12-24,26142.099609
1232,2025-12-26,26042.300781
1233,2025-12-29,25942.099609


In [14]:
index_cons = pd.read_csv(base_dir + index_cons_file, parse_dates=['Date'], usecols=['Date', 'Security Symbol'])
index_cons.insert(1, 'Index_name', 'Nifty 50')
index_cons

,Date,Index_name,Security Symbol
0,2021-03-31,Nifty 50,ADANIPORTS
1,2021-03-31,Nifty 50,JSWSTEEL
2,2021-03-31,Nifty 50,KOTAKBANK
3,2021-03-31,Nifty 50,LT
4,2021-03-31,Nifty 50,M&M
...,...,...,...
495,2025-09-30,Nifty 50,HINDALCO
496,2025-09-30,Nifty 50,HINDUNILVR
497,2025-09-30,Nifty 50,ICICIBANK
498,2025-09-30,Nifty 50,COALINDIA


In [25]:
price_data = pd.read_csv(base_dir + 'all_price_data.csv', parse_dates=['Date'])
price_data

,Symbol,Date,Close,Volume
0,BHARTIARTL,2020-01-01,433.316467,5154996
1,BHARTIARTL,2020-01-02,435.132721,4933053
2,BHARTIARTL,2020-01-03,435.037109,5154587
3,BHARTIARTL,2020-01-06,429.827362,7538915
4,BHARTIARTL,2020-01-07,425.477997,4353883
...,...,...,...,...
84852,M&M,2025-12-24,3636.699951,842897
84853,M&M,2025-12-25,3636.699951,0
84854,M&M,2025-12-26,3623.100098,829910
84855,M&M,2025-12-29,3592.100098,1034692
